## Data Quality Checking

In [1]:
import sys
import pandas as pd
sys.path.append('../')
import pandas as pd
from scripts.db_utils import connect, sql_to_dataframe
from src.data_quality_checks import check_missing_data, check_duplicates, check_data_types, check_numeric_anomalies, get_numeric_columns, get_total_missing_percentage

In [2]:
#opening the connection
conn = connect()

query = """ SELECT * FROM public.xdr_data  """

#loading our dataframe
df = sql_to_dataframe(conn, query)

#closing the connection
conn.close()

# Let’s see if we loaded the df successfully
df.head()

Connecting..
All good, Connection successful!


,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [3]:
# checking for missing value in each columns
missing_data_summary = check_missing_data(df)
print(missing_data_summary)

                                 Column Name  Missing Values
0                                  Bearer Id             991
1                                      Start               1
2                                   Start ms               1
3                                        End               1
4                                     End ms               1
5                                  Dur. (ms)               1
6                                       IMSI             570
7                              MSISDN/Number            1066
8                                       IMEI             572
9                         Last Location Name            1153
10                           Avg RTT DL (ms)           27829
11                           Avg RTT UL (ms)           27812
12                   Avg Bearer TP DL (kbps)               1
13                   Avg Bearer TP UL (kbps)               1
14               TCP DL Retrans. Vol (Bytes)           88146
15               TCP UL 

In [4]:
# Calculate total percentage of missing values
missing_data_percentage = get_total_missing_percentage(df)
print(f"Total Percentage of Missing Values: {missing_data_percentage:.2f}%")

Total Percentage of Missing Values: 12.50%


In [5]:
# checking for duplicated rows in the datasets
duplicate_rows = check_duplicates(df)
print(duplicate_rows)

Success: No duplicated values.


In [6]:
# checking for data type issues per each columns
dtypes_summary = check_data_types(df)
print(dtypes_summary)

Success: Data types per column are uniform.


In [7]:
# list all numberical columns 
numeric_columns = get_numeric_columns(df)
print(numeric_columns)

['Bearer Id', 'Start ms', 'End ms', 'Dur. (ms)', 'IMSI', 'MSISDN/Number', 'IMEI', 'Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)', 'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)', 'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)', '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)', 'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)', '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)', 'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Activity Duration DL (ms)', 'Activity Duration UL (ms)', 'Dur. (ms).1', 'Nb of sec with 125000B < Vol DL', 'Nb of sec with 1250B < Vol UL < 6250B', 'Nb of sec with 31250B < Vol DL < 125000B', 'Nb of sec with 37500B < Vol UL', 'Nb of sec with 6250B < Vol DL < 31250B', 'Nb of sec with 6250B < Vol UL < 37500B', 'Nb of sec with Vol DL < 6250B', 'Nb of sec with Vol UL < 1250B', 'Social Media DL (Bytes)', 'Social Media UL (Bytes)', 'Google DL (Bytes)', 'Google UL (Bytes)', 'Email DL (Bytes)', 'Email 

In [8]:
# checking for anomalies in all numeric columns
for numeric_column in numeric_columns:
    numeric_anomalies = check_numeric_anomalies(df, numeric_column, lower_bound=0, upper_bound=None)
    print(numeric_anomalies)

Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success: No anomalies detected.
Success:

## Data Preprocessing

### Data Cleaning

In [9]:
from sklearn.impute import SimpleImputer
import numpy as np

In [10]:
# Calculate total percentage of missing values
missing_data_percentage = get_total_missing_percentage(df)
print(f"Total Percentage of Missing Values before cleaning: {missing_data_percentage}%")

Total Percentage of Missing Values before cleaning: 12.501637867868759%


In [11]:
# Separate numerical and categorical columns
numerical_columns = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_columns = df.select_dtypes(exclude=[np.number]).columns.tolist()

In [12]:
# Drop columns with all missing values
df = df.dropna(axis=1, how='all')

In [13]:
# Convert numerical columns to the correct data type
for col in numerical_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [14]:
# Impute missing values for numerical columns using mean strategy
num_imputer = SimpleImputer(strategy='mean')
df[numerical_columns] = num_imputer.fit_transform(df[numerical_columns])

In [15]:
# Impute missing values for categorical columns using most frequent strategy (mode)
cat_imputer = SimpleImputer(strategy='most_frequent')
df[categorical_columns] = cat_imputer.fit_transform(df[categorical_columns])

In [16]:
# Calculate total percentage of missing values
missing_data_percentage = get_total_missing_percentage(df)
print(f"Total Percentage of Missing Values after cleaning: {missing_data_percentage}%")

Total Percentage of Missing Values after cleaning: 0.0278664808901274%


### Selecting Relevant columns

In [39]:
print(df.columns.tolist())

['Bearer Id', 'Start', 'Start ms', 'End', 'End ms', 'Dur. (ms)', 'IMSI', 'MSISDN/Number', 'IMEI', 'Last Location Name', 'Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)', 'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)', 'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)', '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)', 'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)', '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)', 'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Activity Duration DL (ms)', 'Activity Duration UL (ms)', 'Dur. (ms).1', 'Handset Manufacturer', 'Handset Type', 'Nb of sec with 125000B < Vol DL', 'Nb of sec with 1250B < Vol UL < 6250B', 'Nb of sec with 31250B < Vol DL < 125000B', 'Nb of sec with 37500B < Vol UL', 'Nb of sec with 6250B < Vol DL < 31250B', 'Nb of sec with 6250B < Vol UL < 37500B', 'Nb of sec with Vol DL < 6250B', 'Nb of sec with Vol UL < 1250B', 'Social Media DL (Bytes)', 'Social Media UL (

In [40]:
# Create a new DataFrame with selected columns
selected_columns = ['Total DL (Bytes)', 'Total UL (Bytes)',
                    'Handset Manufacturer', 'Handset Type',
                    'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)', 'Avg RTT DL (ms)', 'Avg RTT UL (ms)',
                    'Dur. (ms)',
                    'Last Location Name',
                    'Social Media DL (Bytes)', 'Youtube DL (Bytes)', 'Email DL (Bytes)', 'Gaming DL (Bytes)', 'Other DL (Bytes)', 'Other UL (Bytes)']

df_relevant = df[selected_columns]

### Data Summary

In [41]:
# Simple Tabulation for Handset Manufacturer
print("--- Simple Tabulation for Handset Manufacturer ---")
counts_manufacturer = df_relevant['Handset Manufacturer'].value_counts()
total_manufacturer = counts_manufacturer.sum()
percentages_manufacturer = counts_manufacturer.apply(lambda x: round((x / total_manufacturer) * 100, 2))

result_manufacturer = pd.DataFrame({'Handset Manufacturer': counts_manufacturer.index, 'Count': counts_manufacturer.values, 'Percentage': percentages_manufacturer.values})
print(result_manufacturer)
print("\n")
result_manufacturer.head(10)

--- Simple Tabulation for Handset Manufacturer ---
                 Handset Manufacturer  Count  Percentage
0                               Apple  59565       39.86
1                             Samsung  40839       27.33
2                              Huawei  34423       23.04
3                           undefined   8987        6.01
4       Sony Mobile Communications Ab    980        0.66
..                                ...    ...         ...
165              Pt. Arga Mas Lestari      1        0.00
166   Doke Communication (Hk) Limited      1        0.00
167  Vastking Technology (Hk) Limited      1        0.00
168                      Mobiwire Sas      1        0.00
169                 Shenzhen Gaoxinqi      1        0.00

[170 rows x 3 columns]




,Handset Manufacturer,Count,Percentage
0,Apple,59565,39.86
1,Samsung,40839,27.33
2,Huawei,34423,23.04
3,undefined,8987,6.01
4,Sony Mobile Communications Ab,980,0.66
5,Wiko Global Sasu,747,0.50
6,Xiaomi Communications Co Ltd,397,0.27
7,Oneplus Technology (Shenzhen) Co Ltd,342,0.23
8,Asustek,285,0.19
9,Lenovo,254,0.17


In [45]:
# Simple Tabulation for Handset Type
print("--- Simple Tabulation for Handset Type ---")
counts_type = df_relevant['Handset Type'].value_counts()
total_type = counts_type.sum()
percentages_type = counts_type.apply(lambda x: round((x / total_type) * 100, 2))

result_type = pd.DataFrame({'Handset Type': counts_type.index, 'Count': counts_type.values, 'Percentage': percentages_type.values})
print(result_type)
print("\n")
result_type.head(50)

--- Simple Tabulation for Handset Type ---
                                           Handset Type  Count  Percentage
0                                      Huawei B528S-23A  19752       13.22
1                               Apple iPhone 6S (A1688)   9419        6.30
2                                Apple iPhone 6 (A1586)   9023        6.04
3                                             undefined   8987        6.01
4                                Apple iPhone 7 (A1778)   6326        4.23
...                                                 ...    ...         ...
1391          Tct Mobile Suzho. Alcatel One Touch Y580D      1        0.00
1392                          Shenzhen Fortune. M-Ppas6      1        0.00
1393                     Wiko Global Sasu Wiko Ridge 4G      1        0.00
1394  Hongkong Ipro Te. Ipro I9350 I9400 S3 Wave 3.5...      1        0.00
1395             Shenzhen Jeko Co. Blackview Bv5800 Pro      1        0.00

[1396 rows x 3 columns]




,Handset Type,Count,Percentage
0,Huawei B528S-23A,19752,13.22
1,Apple iPhone 6S (A1688),9419,6.30
2,Apple iPhone 6 (A1586),9023,6.04
3,undefined,8987,6.01
4,Apple iPhone 7 (A1778),6326,4.23
5,Apple iPhone Se (A1723),5187,3.47
6,Apple iPhone 8 (A1905),4993,3.34
7,Apple iPhone Xr (A2105),4568,3.06
8,Samsung Galaxy S8 (Sm-G950F),4520,3.02
9,Apple iPhone X (A1901),3813,2.55


In [43]:
# Simple Tabulation for Last Location Name
print("--- Simple Tabulation for Last Location Name ---")
counts_location = df_relevant['Last Location Name'].value_counts()
total_location = counts_location.sum()
percentages_location = counts_location.apply(lambda x: round((x / total_location) * 100, 2))

result_location = pd.DataFrame({'Last Location Name': counts_location.index, 'Count': counts_location.values, 'Percentage': percentages_location.values})
print(result_location)
print("\n")
result_location.head(10)

--- Simple Tabulation for Last Location Name ---
          Last Location Name  Count  Percentage
0                    D41377B     80        0.05
1                    D17085A     59        0.04
2                    D72396C     57        0.04
3      9.16456698599501E+015     55        0.04
4                    D73604C     55        0.04
...                      ...    ...         ...
45542                T79480A      1        0.00
45543                T10447C      1        0.00
45544                T37799B      1        0.00
45545                T79346B      1        0.00
45546                L88342B      1        0.00

[45547 rows x 3 columns]




,Last Location Name,Count,Percentage
0,D41377B,80,0.05
1,D17085A,59,0.04
2,D72396C,57,0.04
3,9.16456698599501E+015,55,0.04
4,D73604C,55,0.04
5,D11119B,54,0.04
6,L41377B,51,0.03
7,9.1645670130491E+015,51,0.03
8,D11002B,50,0.03
9,T11002B,46,0.03


In [44]:
# Numerical variables descriptive summary
numerical_columns = ['Total DL (Bytes)', 'Total UL (Bytes)',
                     'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)', 'Avg RTT DL (ms)', 'Avg RTT UL (ms)',
                     'Dur. (ms)',
                     'Social Media DL (Bytes)', 'Youtube DL (Bytes)', 'Email DL (Bytes)', 'Gaming DL (Bytes)', 'Other UL (Bytes)', 'Other UL (Bytes)']

df_descriptions = df_relevant[numerical_columns].describe()
df_descriptions.head(20)

,Total DL (Bytes),Total UL (Bytes),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),Avg RTT DL (ms),Avg RTT UL (ms),Dur. (ms),Social Media DL (Bytes),Youtube DL (Bytes),Email DL (Bytes),Gaming DL (Bytes),Other UL (Bytes),Other UL (Bytes)
count,1.500010e+05,1.500010e+05,150001.000000,150001.000000,150001.000000,150001.000000,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05
mean,4.546434e+08,4.112121e+07,13300.045927,1770.428647,109.795706,17.662883,1.046086e+05,1.795322e+06,1.163407e+07,1.791729e+06,4.220447e+08,8.264799e+06,8.264799e+06
std,2.441421e+08,1.127635e+07,23971.798635,4625.340082,559.342620,76.529933,8.103735e+04,1.035482e+06,6.710569e+06,1.035840e+06,2.439675e+08,4.769004e+06,4.769004e+06
min,7.114041e+06,2.866892e+06,0.000000,0.000000,0.000000,0.000000,7.142000e+03,1.200000e+01,5.300000e+01,1.400000e+01,2.516000e+03,1.480000e+02,1.480000e+02
25%,2.431072e+08,3.322203e+07,43.000000,47.000000,35.000000,3.000000,5.744200e+04,8.991480e+05,5.833501e+06,8.927930e+05,2.104733e+08,4.145943e+06,4.145943e+06
50%,4.558409e+08,4.114324e+07,63.000000,63.000000,54.000000,7.000000,8.639900e+04,1.794369e+06,1.161602e+07,1.793505e+06,4.234081e+08,8.267071e+06,8.267071e+06
75%,6.657051e+08,4.903424e+07,19710.000000,1120.000000,109.795706,17.662883,1.324300e+05,2.694938e+06,1.744852e+07,2.689327e+06,6.331742e+08,1.238415e+07,1.238415e+07
max,9.029696e+08,7.833131e+07,378160.000000,58613.000000,96923.000000,7120.000000,1.859336e+06,3.586064e+06,2.325910e+07,3.586146e+06,8.434419e+08,1.655882e+07,1.655882e+07


In [49]:
# Function to list top 5 handset types for a given manufacturer
def top_handsets_for_manufacturer(manufacturer):
    df_manufacturer = df_relevant[df_relevant['Handset Manufacturer'] == manufacturer]
    top_handsets = df_manufacturer['Handset Type'].value_counts().head(5)
    top_handsets_percentage = (top_handsets / top_handsets.sum()) * 100
    result_top_handsets = pd.DataFrame({'Handset Type': top_handsets.index, 'Count': top_handsets.values, 'Percentage': top_handsets_percentage.values})
    return result_top_handsets

# Example usage
manufacturer = 'Samsung'  # Enter the top manufacturer here
top_handsets_df = top_handsets_for_manufacturer(manufacturer)
print(f"Top 5 Handsets for {manufacturer}:")
print(top_handsets_df)

Top 5 Handsets for Samsung:
                   Handset Type  Count  Percentage
0  Samsung Galaxy S8 (Sm-G950F)   4520   24.271063
1    Samsung Galaxy A5 Sm-A520F   3724   19.996778
2   Samsung Galaxy J5 (Sm-J530)   3696   19.846426
3   Samsung Galaxy J3 (Sm-J330)   3484   18.708049
4  Samsung Galaxy S7 (Sm-G930X)   3199   17.177684


In [50]:
manufacturer = 'Apple'  # Enter the top manufacturer here
top_handsets_df = top_handsets_for_manufacturer(manufacturer)
print(f"Top 5 Handsets for {manufacturer}:")
print(top_handsets_df)

Top 5 Handsets for Apple:
              Handset Type  Count  Percentage
0  Apple iPhone 6S (A1688)   9419   26.951471
1   Apple iPhone 6 (A1586)   9023   25.818359
2   Apple iPhone 7 (A1778)   6326   18.101179
3  Apple iPhone Se (A1723)   5187   14.842051
4   Apple iPhone 8 (A1905)   4993   14.286941


In [51]:
manufacturer = 'Huawei'  # Enter the top manufacturer here
top_handsets_df = top_handsets_for_manufacturer(manufacturer)
print(f"Top 5 Handsets for {manufacturer}:")
print(top_handsets_df)

Top 5 Handsets for Huawei:
                     Handset Type  Count  Percentage
0                Huawei B528S-23A  19752   75.019940
1                    Huawei E5180   2079    7.896236
2  Huawei P20 Lite Huawei Nova 3E   2021    7.675947
3                      Huawei P20   1480    5.621178
4                  Huawei Y6 2018    997    3.786699
